Extract the Main Zip File

In [2]:
import zipfile
import os

def extract_main_zip(main_zip_file, extract_to_folder):
    # Open the main zip file
    with zipfile.ZipFile(main_zip_file, 'r') as main_zip:
        # Extract all contents of the main zip file
        main_zip.extractall(extract_to_folder)
        print(f"Extracted {main_zip_file} to {extract_to_folder}")


main_zip = 'G:/College/University of Montana/Semester 3/Applied Data Analytics/Wedge Project/Wedge-Project-ADA-Riley-ORorke/data/WedgeZipOfZips(raw).zip'
extract_folder = 'G:/College/University of Montana/Semester 3/Applied Data Analytics/Wedge Project/Wedge-Project-ADA-Riley-ORorke/data/extracted_main_zip'

extract_main_zip(main_zip, extract_folder)


Extracted G:/College/University of Montana/Semester 3/Applied Data Analytics/Wedge Project/Wedge-Project-ADA-Riley-ORorke/data/WedgeZipOfZips(raw).zip to G:/College/University of Montana/Semester 3/Applied Data Analytics/Wedge Project/Wedge-Project-ADA-Riley-ORorke/data/extracted_main_zip


Extract the Nested Zip Files

In [6]:
import zipfile
import os

def extract_all_csvs_to_one_folder(extract_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Walk through the extracted folder and look for zip files
    for root, dirs, files in os.walk(extract_folder):
        for file in files:
            if file.endswith('.zip'):
                nested_zip_path = os.path.join(root, file)
                # Extract the nested zip file
                with zipfile.ZipFile(nested_zip_path, 'r') as nested_zip:
                    for zip_info in nested_zip.infolist():
                        if zip_info.filename.endswith('.csv'):
                            output_file_path = os.path.join(output_folder, zip_info.filename)
                            # Check if the CSV file already exists in the output folder
                            if not os.path.exists(output_file_path):
                                # Extract the CSV if it doesn't already exist
                                nested_zip.extract(zip_info, output_folder)
                                print(f"Extracted {zip_info.filename} to {output_folder}")
                            else:
                                print(f"Skipping {zip_info.filename}, already exists.")

# Example usage
extract_folder = 'G:/College/University of Montana/Semester 3/Applied Data Analytics/Wedge Project/Wedge-Project-ADA-Riley-ORorke/extracted_main_zip'
output_folder = 'D:/WedgeExtractedFiles/extracted_csv_files'  # Extract to different drive for space limitations

extract_all_csvs_to_one_folder(extract_folder, output_folder)


